In [298]:
import pandas as pd
import numpy as np
# from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score,confusion_matrix,f1_score,precision_recall_curve

In [299]:
yelp = pd.read_csv('yelp.csv')
yelp.head()

,business_id,date,review_id,stars,text,type,user_id,cool,useful,funny
0,9yKzy9PApeiPPOUJEtnvkg,1/26/2011,fWKvX83p0-ka4JS3dc6E5A,5,My wife took me here on my birthday for breakf...,review,rLtl8ZkDX5vH5nAx9C3q5Q,2,5,0
1,ZRJwVLyzEJq1VAihDhYiow,7/27/2011,IjZ33sJrzXqU-0X6U8NwyA,5,I have no idea why some people give bad review...,review,0a2KyEL0d3Yb1V6aivbIuQ,0,0,0
2,6oRAC4uyJCsJl1X0WZpVSA,6/14/2012,IESLBzqUCLdSzSqm0eCSxQ,4,love the gyro plate. Rice is so good and I als...,review,0hT2KtfLiobPvh6cDC8JQg,0,1,0
3,_1QQZuf4zZOyFCvXc0o6Vg,5/27/2010,G-WvGaISbqqaMHlNnByodA,5,"Rosie, Dakota, and I LOVE Chaparral Dog Park!!...",review,uZetl9T0NcROGOyFfughhg,1,2,0
4,6ozycU1RpktNG2-1BroVtw,1/5/2012,1uJFq2r5QfJG_6ExMRCaGw,5,General Manager Scott Petello is a good egg!!!...,review,vYmM4KTsC8ZfQBg-j5MWkw,0,0,0


In [300]:
yelp.shape

(10000, 10)

In [301]:
yelp.stars.value_counts()

4    3526
5    3337
3    1461
2     927
1     749
Name: stars, dtype: int64

In [302]:
print(yelp.cool.value_counts().head())
print(yelp.useful.value_counts().head())
print(yelp.funny.value_counts().head())


0    6290
1    1955
2     749
3     396
4     209
Name: cool, dtype: int64
0    4130
1    2848
2    1323
3     711
4     335
Name: useful, dtype: int64
0    7013
1    1632
2     603
3     271
4     161
Name: funny, dtype: int64


best reviews

In [303]:
yelp_best_worst = yelp[(yelp.stars==5)|(yelp.stars==1)]
yelp_best_worst.head()

,business_id,date,review_id,stars,text,type,user_id,cool,useful,funny
0,9yKzy9PApeiPPOUJEtnvkg,1/26/2011,fWKvX83p0-ka4JS3dc6E5A,5,My wife took me here on my birthday for breakf...,review,rLtl8ZkDX5vH5nAx9C3q5Q,2,5,0
1,ZRJwVLyzEJq1VAihDhYiow,7/27/2011,IjZ33sJrzXqU-0X6U8NwyA,5,I have no idea why some people give bad review...,review,0a2KyEL0d3Yb1V6aivbIuQ,0,0,0
3,_1QQZuf4zZOyFCvXc0o6Vg,5/27/2010,G-WvGaISbqqaMHlNnByodA,5,"Rosie, Dakota, and I LOVE Chaparral Dog Park!!...",review,uZetl9T0NcROGOyFfughhg,1,2,0
4,6ozycU1RpktNG2-1BroVtw,1/5/2012,1uJFq2r5QfJG_6ExMRCaGw,5,General Manager Scott Petello is a good egg!!!...,review,vYmM4KTsC8ZfQBg-j5MWkw,0,0,0
6,zp713qNhx8d9KCJJnrw1xA,2/12/2010,riFQ3vxNpP4rWLk_CSri2A,5,Drop what you're doing and drive here. After I...,review,wFweIWhv2fREZV_dYkz_1g,7,7,4


In [304]:
yelp_best_worst.shape

(4086, 10)

modeling

In [305]:
X = yelp_best_worst.text
y = yelp_best_worst.stars

In [306]:
from sklearn.cross_validation import train_test_split

In [307]:
X_train,X_test,y_train,y_test = train_test_split(X,y,random_state=1)

In [308]:
X_train.shape,y_train.shape,X_test.shape,y_test.shape

((3064,), (3064,), (1022,), (1022,))

In [309]:
from sklearn.feature_extraction.text import TfidfVectorizer,CountVectorizer
vect = CountVectorizer()

creating document term matrix

In [310]:
X_train_dtm = vect.fit_transform(X_train)
X_train_dtm.shape

(3064, 16825)

In [311]:
X_test_dtm = vect.transform(X_test)
X_test_dtm.shape

(1022, 16825)

In [312]:
from sklearn.naive_bayes import MultinomialNB
mnb = MultinomialNB()

In [313]:
mnb.fit(X_train_dtm,y_train)

MultinomialNB(alpha=1.0, class_prior=None, fit_prior=True)

In [314]:
y_pred_class = mnb.predict(X_test_dtm)

In [315]:
from sklearn.metrics import accuracy_score
accuracy_score(y_test,y_pred_class)

0.91878669275929548

In [316]:
confusion_matrix(y_test,y_pred_class)

array([[126,  58],
       [ 25, 813]], dtype=int64)

In [317]:
pred_proba = mnb.predict_proba(X_test_dtm)
pred_proba

array([[  6.21708191e-13,   1.00000000e+00],
       [  2.40125920e-04,   9.99759874e-01],
       [  2.04567380e-10,   1.00000000e+00],
       ..., 
       [  2.86600345e-13,   1.00000000e+00],
       [  1.00000000e+00,   1.05200807e-19],
       [  8.46839316e-05,   9.99915316e-01]])

In [318]:
y_test.value_counts()

5    838
1    184
Name: stars, dtype: int64

In [319]:
y_test.value_counts().head(1) / y_test.shape #null accuracy

5    0.819961
Name: stars, dtype: float64

examining the confusion matrix FP and FN values

In [320]:
X_test[y_test > y_pred_class].shape #these are FN values

(25,)

In [321]:
X_test[y_test > y_pred_class].head()

7148    I now consider myself an Arizonian. If you dri...
4963    This is by far my favourite department store, ...
6318    Since I have ranted recently on poor customer ...
380     This is a must try for any Mani Pedi fan. I us...
5565    I`ve had work done by this shop a few times th...
Name: text, dtype: object

In [322]:
X_test[y_test < y_pred_class].shape #FP values

(58,)

In [323]:
X_test[y_test < y_pred_class].head()

2175    This has to be the worst restaurant in terms o...
1781    If you like the stuck up Scottsdale vibe this ...
2674    I'm sorry to be what seems to be the lone one ...
9984    Went last night to Whore Foods to get basics t...
3392    I found Lisa G's while driving through phoenix...
Name: text, dtype: object

In [324]:
X_train_tokens = vect.get_feature_names()
len(X_train_tokens)

16825

In [325]:
# store the number of times each token appears across each class
one_star_token_count = mnb.feature_count_[0, :]
five_star_token_count = mnb.feature_count_[1, :]

In [326]:
# create a DataFrame of tokens with their separate one-star and five-star counts
tokens = pd.DataFrame({'token':X_train_tokens, 'one_star':one_star_token_count, 'five_star':five_star_token_count}).set_index('token')

we have taken only 2 classes, now we will take all 5 classes which is multi class classification

In [327]:
X = yelp.text
y = yelp.stars

In [328]:
y.value_counts()

4    3526
5    3337
3    1461
2     927
1     749
Name: stars, dtype: int64

In [329]:
# split X and y into training and testing sets
X_train, X_test, y_train, y_test = train_test_split(X, y, random_state=1)

In [330]:
# create document-term matrices using CountVectorizer
X_train_dtm = vect.fit_transform(X_train)
X_test_dtm = vect.transform(X_test)

In [331]:
# fit a Multinomial Naive Bayes model
mnb.fit(X_train_dtm, y_train)

MultinomialNB(alpha=1.0, class_prior=None, fit_prior=True)

In [332]:
# make class predictions
y_pred_class = mnb.predict(X_test_dtm)

In [333]:
# calculate the accuary
accuracy_score(y_test, y_pred_class)

0.47120000000000001

In [334]:
confusion_matrix(y_test,y_pred_class)

array([[ 55,  14,  24,  65,  27],
       [ 28,  16,  41, 122,  27],
       [  5,   7,  35, 281,  37],
       [  7,   0,  16, 629, 232],
       [  6,   4,   6, 373, 443]], dtype=int64)

In [335]:
from sklearn.metrics import classification_report

In [336]:
print(classification_report(y_test,y_pred_class))

             precision    recall  f1-score   support

          1       0.54      0.30      0.38       185
          2       0.39      0.07      0.12       234
          3       0.29      0.10      0.14       365
          4       0.43      0.71      0.53       884
          5       0.58      0.53      0.55       832

avg / total       0.46      0.47      0.43      2500



higher precision = higher confidence

In [337]:
precsiion_1 = 55/(55+28+5+7+6)
precsiion_1

0.5445544554455446

In [338]:
recall_1 = 55/(55+14+24+65+27)
recall_1

0.2972972972972973

**F1 score** is a weighted average of precision and recall.

In [339]:
# manually calculate the F1 score for class 1
f1 = 2 * (precsiion_1 * recall_1) / (precsiion_1 + recall_1)
print(f1)

0.38461538461538464
